In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import pymongo

In [2]:
# Initializing a variable with the path for chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# Creating a Browser instance for Google Chrome using chromedriver
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Initializing a variable with the url of the AAA website that displays fuel prices by grade by state
fuel_price_url = "https://gasprices.aaa.com/state-gas-price-averages/"
# Visiting fuel_price_url
browser.visit(fuel_price_url)

In [4]:
# Initializing a variable with the html script of fuel_price_url
html = browser.html
# Initializing a variable with the html script parsed by BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
# Initializing a variable with the list created from the html script for the fuel price table
table = pd.read_html(html)

In [5]:
# Intitalizing a DataFrame from the list "table"
state_fuel_df = table[0]
# Setting the column names of state_fuel_df
state_fuel_df.columns = ["state", "regular", "mid_grade", "premium", "diesel"]
# Displaying state_fuel_df
state_fuel_df.head()

,state,regular,mid_grade,premium,diesel
0,Alaska,$3.064,$3.160,$3.303,$2.982
1,Alabama,$2.300,$2.628,$2.949,$2.790
2,Arkansas,$2.310,$2.606,$2.871,$2.723
3,Arizona,$2.781,$3.009,$3.223,$3.015
4,California,$3.609,$3.777,$3.886,$3.918


In [6]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [7]:
# Define database and collection
db = client.etl_db
collection = db.fuel_prices
db.fuel_prices.drop()

In [8]:
# Using a for-loop to iterate through the rows of state_fuel_df and insert each state's name and its fuel prices into 
# the MongoDB database fuel_prices_db collection fuel_prices as individual documents
for index, row in state_fuel_df.iterrows():

    post = {
                "state": row["state"],
                "regular": row["regular"],
                "mid_grade": row["mid_grade"],
                "premium": row["premium"],
                "diesel": row["diesel"]
            }

    collection.insert_one(post)
